In [ ]:
import sys
!conda install -c conda-forge --yes --prefix {sys.prefix} pygeos

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import hvplot.pandas
import hvplot.xarray

In [ ]:
# Datasets
LTS = gpd.read_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\LUCAS_TOPSOIL_shape\LTS_subset.shp')
BDF5L = gpd.read_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\BFDL_ML46\BFD5L_subset.shp')
NDVI_10Dmax = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\NDVI_10Dmax_1km.nc')
RADOLAN = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\RADOLAN_1km.nc')
SSM = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\SSM_1km_V1.nc')
EVAPO_R = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\EVAPOTRANSPIRATION_REAL.nc')
EVAPO_P = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\EVAPOTRANSPIRATION_POTENTIAL.nc')
L7_sr_ndvi = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\l7_sr_ndvi_100m.nc')
L8_sr_ndvi = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\l8_sr_ndvi_100m.nc')
S2_1C = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\s2_1c.nc')
S2_1C_ndvi = xr.open_dataset(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\s2_1c_ndvi_100m.nc')


In [ ]:
# Define a point of interest (x,y), so (longitude, latitude).
point = (8.51559, 49.7429) #LTS ID 42142960

In [ ]:
# Create a DataFrame from point of interest.
p_df = pd.DataFrame.from_dict({'x':[8.51559], 'y':[49.7429]})

In [ ]:
# Create GeoDataFrame from DataFrame.
p_gdf = gpd.GeoDataFrame(p_df, geometry=gpd.points_from_xy(p_df.x, p_df.y), crs='EPSG:4326')

In [ ]:
## Table data query.
# Query LTS data.
p_gdf = p_gdf.sjoin_nearest(LTS, lsuffix='LTS_left', rsuffix='LTS_right', distance_col='LTS_distance')
# Query BDF5L data.
p_gdf = p_gdf.sjoin_nearest(BDF5L, lsuffix='BDF5L_left', rsuffix='BDF5L_right', distance_col='BDF5L_distance')

In [ ]:
## Grid data query.
test = SSM.sel(lon=point[0], lat=point[1], method='nearest')
test = test.where(test.ssm != np.nan, drop=True)
test.ssm.hvplot.line(groupby='time.year')

In [ ]:
# Plot GeoDataFrame table.
p_gdf.hvplot.table(columns=[x for x in list(p_gdf.columns.values) if x != 'geometry'])